In [ ]:
!sudo apt-get install libpq-dev
!pip install psycopg2

In [ ]:
import psycopg2
import pandas as pd

In [57]:
# Replace these with your actual database credentials
db_params = {
    'host': '192.168.51.197',
    # 'host': '127.0.0.1',
    'database': 'smis.pro',
    'user': 'postgres',
    'password': 'helloadmin123',
    'port': '5432'
}

# Establish a connection
conn = psycopg2.connect(**db_params)

# Create a cursor object
cursor = conn.cursor()

In [58]:
cursor.execute("SELECT id_card, photo FROM students where concat(id_card,'.jpg') != photo and id_card like 'p2023%';")
data = cursor.fetchall()

cursor.close()

In [59]:
df = pd.DataFrame(data, columns=['id_card', 'photo'])
df

,id_card,photo
0,p20230034,2024/0140.jpg
1,p20230033,2024/0457.jpg
2,p20230036,2024/0542.jpg
3,p20230035,2024/1593.jpg
4,p20230011,2024/1501.jpg
5,p20230015,2024/1621.jpg
6,p20230021,2024/2219.jpg
7,p20230018,2024/2984.jpg
8,p20230024,2024/3710.jpg
9,p20230020,2024/1459.jpg


In [55]:
import subprocess
from urllib.request import urlretrieve
from queue import Queue
import threading

id_failed = {"successed": [], "failed":[]}
dirs = "photos/int"

# urlimage = "https://smisimg.itc.edu.kh/img/profiles"
urlimage = "https://etudiant.itc.edu.kh/storage/candidates/images"

# Function to download a file
def download_file(photo, id_card):
    try:
        photo = id_card.replace("p2023","2024")
        # print(f"{urlimage}/{photo}.png")
        urlretrieve(f"{urlimage}/{photo}.png", f"./{dirs}/{id_card}.jpg")
        # subprocess.run(['ln', '-s', f'./{dirs}/{id_card}.jpg',f'./{dirs}/{photo}'],check=True)
        id_failed['successed'].append(photo)
    except Exception as e:
        id_failed['failed'].append(photo)

In [56]:
# Number of worker threads
num_threads = 3

# Create a queue to manage the download tasks
download_queue = Queue()

# Worker function to process the download queue
def worker():
    while True:
        values = download_queue.get()
        if values is None:
            break
        download_file(values["photo"], values["id_card"])
        download_queue.task_done()

# Create and start worker threads
threads = []
for _ in range(num_threads):
    thread = threading.Thread(target=worker)
    thread.start()
    threads.append(thread)

# Add URLs to the download queue
for index, row in df.iterrows():
    download_queue.put(row)

# Wait for all download tasks to be completed
download_queue.join()

# Stop worker threads by adding None to the queue
for _ in range(num_threads):
    download_queue.put(None)

# Wait for all worker threads to finish
for thread in threads:
    thread.join()

print("All downloads completed.")

All downloads completed.


In [ ]:
for index, row in df.iterrows():
    # print(row['photo'], row['id_card'])
    download_file(row['photo'],row['id_card'])

In [ ]:
import subprocess
from getpass import getpass

# password = getpass("Enter your SSH password: ")

path = "/var/www/html/smis/public/img/profiles"
for i in df.values:
    base_path = f'scp -r {path}/{i[1]} {path}/{i[0]}.jpg'
    script = f'ssh root@192.168.51.88 "{base_path}"'
    try:
        subprocess.run(script, shell=True, check=True)
        print(f"Script {i[1]} -> {i[0]} executed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error executing script {i[1]}. Details: {e}")